## Map Figshare metadata to another schema
This is just an example.

## Import libraries

In [1]:
import json
import requests
import pandas as pd
import datetime

## Set Base URL

In [2]:
#Set the base URL
BASE_URL = 'https://api.figshare.com/v2'

## Create list of ids
You can get this through a search, from the Administrative Batch Management tool, etc. Or hand enter it

In [7]:
item_ids = [18621923,23551983,23284910]

## Download the metadata and map it to new fields

In [8]:
formatted = []

for item in item_ids:
    m = requests.get(BASE_URL + '/articles/' + str(item))
    metadata=json.loads(m.text)
    
    if m.status_code == 200: #if the record is not a previously published and then unpublished record
        
        #Process the author metadata
        counter = 0
        author_list = ""
        author_count = len(metadata['authors'])
        for name in metadata['authors']: #Format author list to be readable
            if counter == 0:
                author_list = author_list + name['full_name']
                counter += 1
            elif counter < author_count:
                author_list = author_list + ', ' + name['full_name']
                counter += 1
        
        #Get categories
        cats = []
        for c in metadata['categories']:
            cats.append(c['title'])

        mapped = dict(
        creators = author_list,
        date = metadata['timeline']['posted'],
        description = metadata['description'],
        source = ['Harvested Record'],
        identifier = metadata['doi'],
        rights = metadata['license']['name'],
        subject = [cats],
        titles = metadata['title'],
        types = metadata['defined_type_name']
        )
        
        formatted.append(mapped)

In [9]:
formatted

[{'creators': 'ICES',
  'date': '2023-06-23T10:50:52',
  'description': '<h4>Corrigendum (23/06/2023): \xa0An error in the data used to support primary production estimates was detected and corrected in 2022. Increasing trends in net primary production reported in the highlights, in the graphical summary and in the phytoplankton section are erroneous. Readers should refer to the WGINOR 2023 report (ICES, 2023; https://doi.org/10.17895/ices.pub.22110260) where the error has been corrected.</h4>\n<p>The task of the Working Group on Integrated Ecosystem Assessments for the Norwegian Sea (WGINOR) is to advance the understanding of the Norwegian Sea ecosystem and to develop an operational approach for integrated ecosystem assessment (IEA) that is applicable to manage-ment. This includes research on functional connections within the ecosystem, compiling relevant time series and develop models suitable for IEA. The current report contains the results from the second meeting of a 3-year term w

## Export the records as a CSV


In [ ]:
save_file = formatted.to_csv('formatted-metadata.csv',encoding='utf-8', index=None)